In [12]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions 
from selenium.common.exceptions import NoSuchElementException
import os
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
import pandas as pd
import time
import zipfile
from datetime import date
from datetime import timedelta  


In [17]:
dados = pd.read_csv("C:/Users/Giulio/Downloads/df_info.csv")
p = 0
for x in dados['name.company']:
    if x == 'PETRÓLEO BRASILEIRO  S.A.  - PETROBRAS':
        break
    p+=1
dados.drop(p, inplace = True)
dados.index = range(0, len(dados))

In [18]:
profile = webdriver.FirefoxProfile()
profile.set_preference('browser.download.folderList', "C:/Users/Giulio/dados_financeiros")
profile.set_preference('browser.download.manager.showWhenStarting', False)
profile.set_preference('browser.download.dir', os.getcwd())
profile.set_preference('browser.helperApps.neverAsk.saveToDisk', 'application/zip')
profile.set_preference('browser.helperApps.neverAsk.openFile', 'text/html')
profile.set_preference('general.warnOnAboutConfig', False)
profile.set_preference("dom.disable_open_during_load", False) 
profile.update_preferences()
gecko_path = 'C:/Users/Giulio/webdriver/geckodriver.exe'
driver = webdriver.Firefox(firefox_profile=profile,executable_path=gecko_path)
driver.get("https://www.msperlin.com/shiny/GetDFPData/")

In [19]:
def chamada_1():
    driver.find_element_by_partial_link_text("Download Data").click()
    for i in range(10):
        try:
            driver.find_element_by_xpath(
                ".//*[contains(text(), 'FRE')]"
            ).click()
            driver.find_element_by_xpath(
                ".//*[contains(text(), 'FCA')]"
            ).click()
            break
        except NoSuchElementException as e:
            print('retry in 1s.')
            time.sleep(1)
    else:
        raise e
        
def chamada_2():    
    driver.find_element_by_id("actionid").click()
    wait = WebDriverWait(driver, 15)
    wait.until(expected_conditions.text_to_be_present_in_element(
        (By.ID, "text_action_id"), "DONE. Hit download.."))
    driver.find_element_by_id("text_action_id").click()
    driver.find_element_by_id("downloadData").click()


def chamada_error():
    driver.get("https://www.msperlin.com/shiny/GetDFPData/")
    chamada_1()
    driver.find_element_by_xpath(".//*[contains(text(), 'Select available companies')]").click()

In [20]:
l = []
p = 0
z = 0
data = str(date.today() + timedelta(days=1))
for nomes_cias in dados['name.company']:   
    l.append(nomes_cias)
    
    if len(l) == 19:

        if z == 0:
            file = "C:/Users/Giulio/Downloads/GetDFPData_Export_{}.zip".format(data)
        else:
            file = "C:/Users/Giulio/Downloads/GetDFPData_Export_{}({}).zip".format(data, z)
        
        chamada_1()
        driver.find_element_by_xpath(".//*[contains(text(), 'Select available companies')]").click()               
        for nomes in l:
            driver.find_element_by_xpath(".//*[contains(text(), '{}')]".format(nomes)).click()
        try:
            chamada_2()
            time.sleep(5)
            try:
                if type(zipfile.ZipFile(file).namelist()) != list:
                    driver.find_element_by_id("downloadData").click()
                time.sleep(5)
            except:
                driver.find_element_by_id("downloadData").click()
                time.sleep(5)
        
        except:
            driver.get("https://google.com")
            chamada_error()
            for nomes in l:
                driver.find_element_by_xpath(".//*[contains(text(), '{}')]".format(nomes)).click()
            chamada_2()
            time.sleep(5)
            try:
                if type(zipfile.ZipFile(file).namelist()) != list:
                    driver.find_element_by_id("downloadData").click()
                    time.sleep(5)
            except:
                driver.find_element_by_id("downloadData").click()
                time.sleep(5)
                
        driver.refresh()
        time.sleep(5)
                        
        l = []
        z += 1
          
    p+=1
    if p == 324:
        for i in range(0, 5):
            l.append(dados['name.company'].loc[p+i])
        chamada_1()
        driver.find_element_by_xpath(".//*[contains(text(), 'Select available companies')]").click()
        for nomes in l:
            driver.find_element_by_xpath(".//*[contains(text(), '{}')]".format(nomes)).click()
        chamada_2()
        time.sleep(10)
        try:
            if type(zipfile.ZipFile(file).namelist()) != list:
                driver.find_element_by_id("downloadData").click()
                time.sleep(10)
        except:
                driver.find_element_by_id("downloadData").click()
                time.sleep(10)
driver.close()
            